In [0]:
# https://keras.io/
!pip install -q keras
import keras

In [0]:
import keras
from keras.datasets import cifar10
from keras.models import Model, Sequential
from keras.layers import Dense, Dropout, Flatten, Input, AveragePooling2D, merge, Activation
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras.layers import Concatenate
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint

In [0]:
# this part will prevent tensorflow to allocate all the avaliable GPU Memory
# backend
import tensorflow as tf
from keras import backend as k

# Don't pre-allocate memory; allocate as-needed
config = tf.ConfigProto()
config.gpu_options.allow_growth = True

# Create a session with the above options specified.
k.tensorflow_backend.set_session(tf.Session(config=config))

In [0]:
# Hyperparameters
batch_size = 128
num_classes = 10
epochs = 50
l = 20
num_filter = 24
compression = 0.5
dropout_rate = 0.3

In [0]:
# Load CIFAR10 Data
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
img_height, img_width, channel = x_train.shape[1],x_train.shape[2],x_train.shape[3]

# convert to one hot encoing 
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

In [0]:
# Dense Block
def add_denseblock(input, num_filter = 24, dropout_rate = 0.3):
    global compression
    temp = input
    for _ in range(l):
        BatchNorm = BatchNormalization()(temp)
        relu = Activation('relu')(BatchNorm)
        Conv2D_3_3 = Conv2D(int(num_filter*compression), (3,3), use_bias=False ,padding='same')(relu)
        if dropout_rate>0:
          Conv2D_3_3 = Dropout(dropout_rate)(Conv2D_3_3)
        concat = Concatenate(axis=-1)([temp,Conv2D_3_3])
        
        temp = concat
        
    return temp

In [0]:
def add_transition(input, num_filter = 24, dropout_rate = 0.3):
    global compression
    BatchNorm = BatchNormalization()(input)
    relu = Activation('relu')(BatchNorm)
    Conv2D_BottleNeck = Conv2D(int(num_filter*compression), (1,1), use_bias=False ,padding='same')(relu)
    if dropout_rate>0:
      Conv2D_BottleNeck = Dropout(dropout_rate)(Conv2D_BottleNeck)
    avg = AveragePooling2D(pool_size=(2,2))(Conv2D_BottleNeck)
    
    return avg

In [0]:
def output_layer(input):
    global compression
    BatchNorm = BatchNormalization()(input)
    relu = Activation('relu')(BatchNorm)
    AvgPooling = AveragePooling2D(pool_size=(2,2))(relu)
    flat = Flatten()(AvgPooling)
    output = Dense(num_classes, activation='softmax')(flat)
    
    return output

In [0]:
num_filter = 24
dropout_rate = 0.3
l = 20
input = Input(shape=(img_height, img_width, channel,))
First_Conv2D = Conv2D(num_filter, (3,3), use_bias=False ,padding='same')(input)

First_Block = add_denseblock(First_Conv2D, num_filter, dropout_rate)
First_Transition = add_transition(First_Block, num_filter, dropout_rate)

Second_Block = add_denseblock(First_Transition, num_filter, dropout_rate)
Second_Transition = add_transition(Second_Block, num_filter, dropout_rate)

Third_Block = add_denseblock(Second_Transition, num_filter, dropout_rate)
Third_Transition = add_transition(Third_Block, num_filter, dropout_rate)

Last_Block = add_denseblock(Third_Transition,  num_filter, dropout_rate)
output = output_layer(Last_Block)


In [36]:
model = Model(inputs=[input], outputs=[output])
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
conv2d_169 (Conv2D)             (None, 32, 32, 24)   648         input_3[0][0]                    
__________________________________________________________________________________________________
batch_normalization_169 (BatchN (None, 32, 32, 24)   96          conv2d_169[0][0]                 
__________________________________________________________________________________________________
activation_169 (Activation)     (None, 32, 32, 24)   0           batch_normalization_169[0][0]    
__________________________________________________________________________________________________
conv2d_170

__________________________________________________________________________________________________
activation_224 (Activation)     (None, 8, 8, 168)    0           batch_normalization_224[0][0]    
__________________________________________________________________________________________________
conv2d_225 (Conv2D)             (None, 8, 8, 12)     18144       activation_224[0][0]             
__________________________________________________________________________________________________
dropout_222 (Dropout)           (None, 8, 8, 12)     0           conv2d_225[0][0]                 
__________________________________________________________________________________________________
concatenate_214 (Concatenate)   (None, 8, 8, 180)    0           concatenate_213[0][0]            
                                                                 dropout_222[0][0]                
__________________________________________________________________________________________________
batch_norm

In [0]:
path="weights.hdf5"
checkpoint = ModelCheckpoint(path, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]

In [0]:
# determine Loss function and Optimizer
model.compile(loss='categorical_crossentropy',
              optimizer=Adam(),
              metrics=['accuracy'])

In [46]:
model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
          callbacks=callbacks_list,
                    validation_data=(x_test, y_test))

Train on 50000 samples, validate on 10000 samples
Epoch 1/50
50000/50000 [==============================] - 556s 11ms/step - loss: 0.8280 - acc: 0.7075 - val_loss: 1.2558 - val_acc: 0.6438

Epoch 00001: val_acc improved from -inf to 0.64380, saving model to weights.hdf5
Epoch 2/50
14720/50000 [=======>......................] - ETA: 5:49 - loss: 0.7768 - acc: 0.7221

50000/50000 [==============================] - 530s 11ms/step - loss: 0.7605 - acc: 0.7304 - val_loss: 1.5378 - val_acc: 0.6245

Epoch 00002: val_acc did not improve from 0.64380
Epoch 3/50
35456/50000 [====================>.........] - ETA: 2:23 - loss: 0.7099 - acc: 0.7520

50000/50000 [==============================] - 530s 11ms/step - loss: 0.7084 - acc: 0.7520 - val_loss: 1.1243 - val_acc: 0.7053

Epoch 00003: val_acc improved from 0.64380 to 0.70530, saving model to weights.hdf5
Epoch 4/50
37120/50000 [=====================>........] - ETA: 2:07 - loss: 0.6720 - acc: 0.7633

50000/50000 [==============================] - 529s 11ms/step - loss: 0.6652 - acc: 0.7659 - val_loss: 1.6416 - val_acc: 0.6222

Epoch 00004: val_acc did not improve from 0.70530
Epoch 5/50
42368/50000 [========================>.....] - ETA: 1:15 - loss: 0.6304 - acc: 0.7798

50000/50000 [==============================] - 530s 11ms/step - loss: 0.6276 - acc: 0.7809 - val_loss: 1.4591 - val_acc: 0.6493

Epoch 00005: val_acc did not improve from 0.70530
Epoch 6/50
44032/50000 [=========================>....] - ETA: 59s - loss: 0.5965 - acc: 0.7905 

...

50000/50000 [==============================] - 527s 11ms/step - loss: 0.2166 - acc: 0.9228 - val_loss: 0.5934 - val_acc: 0.8498

Epoch 00046: val_acc did not improve from 0.85390
Epoch 47/50
44416/50000 [=========================>....] - ETA: 54s - loss: 0.2126 - acc: 0.9249

50000/50000 [==============================] - 526s 11ms/step - loss: 0.2133 - acc: 0.9247 - val_loss: 0.5656 - val_acc: 0.8606

Epoch 00047: val_acc improved from 0.85390 to 0.86060, saving model to weights.hdf5
Epoch 48/50
39424/50000 [======================>.......] - ETA: 1:44 - loss: 0.2062 - acc: 0.9267

50000/50000 [==============================] - 528s 11ms/step - loss: 0.2086 - acc: 0.9258 - val_loss: 0.7451 - val_acc: 0.8312

Epoch 00048: val_acc did not improve from 0.86060
Epoch 49/50
42880/50000 [========================>.....] - ETA: 1:10 - loss: 0.2087 - acc: 0.9253

50000/50000 [==============================] - 529s 11ms/step - loss: 0.2086 - acc: 0.9258 - val_loss: 0.6913 - val_acc: 0.8335

Epoch 00049: val_acc did not improve from 0.86060
Epoch 50/50
44032/50000 [=========================>....] - ETA: 58s - loss: 0.2014 - acc: 0.9284 

50000/50000 [==============================] - 529s 11ms/step - loss: 0.2004 - acc: 0.9287 - val_loss: 0.8540 - val_acc: 0.8158

Epoch 00050: val_acc did not improve from 0.86060


In [0]:
model.load_weights("weights.hdf5")

In [48]:
# Test the model
score = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

10000/10000 [==============================] - 46s 5ms/step
Test loss: 0.5656300925970078
Test accuracy: 0.8606


In [49]:
# Save the trained weights in to .h5 format
model.save_weights("DNST_model.h5")
print("Saved model to disk")

Saved model to disk


In [0]:
from google.colab import files

files.download('DNST_model.h5')